In [ ]:
import psycopg2
import numpy as np
import json
from gensim import corpora, models, similarities
from gensim.models.doc2vec import LabeledSentence
import re

<h1> Connect to emoji DB </h1>

In [2]:
conn = psycopg2.connect("host=localhost port=5432 dbname=emoji_db user=postgres password=darkmatter")
cur = conn.cursor()

In [3]:
cur.execute("SELECT text from emoji_tweet where lang='en';")
Qresults=cur.fetchall()

<h1> Clean the text </h1>

In [4]:
emj_text=[val[0].replace('.','').replace(',','').replace(';','').replace(':','').replace('!','').replace('?','') for val in Qresults]

In [5]:
#split on period for individual sentences? .... later

In [ ]:
# Remove punnctuation. Words that start with RT,@,#,http
#we will need to split emojis from words too... have functions for that in emoji_class

In [ ]:
emj_words=[[word.lower() for word in text.split() if ('RT' not in word) & ('@' not in word)\
          & ('#' not in word) & ('http' not in word)] for text in emj_text]
emj_words[2]

In [121]:
def most_sim_emj(most_sim):
    sim_emj=[val for val in most_sim if re.findall('[a-z,0-9]',val[0].lower())==[]]
    xdata=[val[0] for val in sim_emj]
    ydata=[val[1] for val in sim_emj]
    return xdata,ydata


<h1> word to vec </h1>

In [135]:
Qmodel = models.Word2Vec(emj_words, size=100, window=10, min_count=2, workers=8)

In [149]:
most_sim_emj(Qmodel.most_similar(u"fun",topn=1000)) #define return emoji in emoji class... given an array

(['😋🍓🍉🍌🍳🍞🍯🍴',
  '😛😘🎉',
  '😂😂😂😂😂😂😭🙄🙄',
  '🍃🍀🌹🌷🌸🍇🍀🍃',
  '💙✨💍',
  '🌹💕🎈🎉',
  '👭🙌',
  '🙈🍻🍻',
  '⚾️🍀',
  '😂💀😂‼️😂‼️',
  '👅💦🐱🍆🍑💦💦😁',
  '☺️☺️😏',
  '💃🏻',
  '🚫👻',
  '🍕🍕🍕🍕🍕🍕🍕🍕🍕',
  '🎳',
  '👫💏💑',
  '😂🎭',
  '⭐️🇮🇹',
  '🙈🙈🙊',
  '😄😨',
  '🙄😳😂',
  '💭👀',
  '😅😜👸🏼',
  '💕✨🔥',
  '🎉🕶',
  '😂😭😭😭😭😭😭😂😂😂😭😂😂😂😭😂😂😂😭😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂',
  '🐦🐶🐐',
  '💛✝',
  '🐼‼️😈❤️😊',
  '😏😍🙈',
  '💆😂',
  '💚💨',
  '🍎🐝',
  '😂😂😣😣',
  '🙉💁🏼',
  '😁😔😂',
  '😬👑',
  '🐣🐣🐥',
  '🙆🏻🙋🏻',
  '💜💗😍',
  '😜🐝🍋',
  '👍🏻😜',
  '😂😛❤️',
  '🍼🚺🚹',
  '😏😏😋',
  '🤖',
  '🙄😌😂',
  '🙏🏽🎀',
  '🍹🍛',
  '😃🌼',
  '🤔🙄🙄🙄🙄',
  '📸👾',
  '💜🐐',
  '😉🌚',
  '😪🏈',
  '✨😆',
  '🏋🏾🏋🏾',
  '🇯🇲🙏🏾🙌🏾🇺🇸',
  '😖😒',
  '🙆🏼💭💗',
  '🙂🤑',
  '👍👍👍👍👍👍👍👍👍',
  '💃🏻🔥',
  '💑🎶🎧🎤',
  '💘👼🏾',
  '🎤🎶👭👯',
  '🙄😩💀',
  '🙈😢💕😻🎶🍃✨💒💍',
  '🚀💫',
  '😌😺',
  '⚾️❤️',
  '💤🌌',
  '🙈😍🙏😭',
  '🙊😀',
  '😁🍑',
  '☺️💪🏽🍑',
  '🎊🎉🎁🎂🎈',
  '🏆‼️',
  '🙆🏻✨',
  '🌲🌧',
  '😈📶',
  '👭👭',
  '🍭🍹',
  '🙍🏻🙍🏻',
  '😊💁🏽😄',
  '😐🙄💯',
  '🎤💏💞',
  '😫😂😂',
  '😆😆😆👍',
  '🏊🏻',
  '💚🎈😘💥',
  '😊❤️😂',
  '🌚🌚🌚🌚🌚🌚',
  '😂🎉🎂',
  '☺️☕️',
  '🐥🌸🎉',
  '🌧💧',
  '🎉🌵',
  '😍🐯📚🎓',
  '😏💪',
  '💪💪👏👏',
  '🔆🌊

In [150]:
Qmodel.save('data/Emoji2vec')

In [79]:
Emoji2vec=models.Word2Vec.load('data/Emoji2vec')

In [80]:
most_sim_emj(Qmodel.most_similar(u"up",topn=5))

['🆙']

In [136]:
len(Qmodel.vocab)

261447

<h1> Train bigrams </h1>

In [51]:
Qbigram = models.Phrases(emj_words,min_count=5) #model bigrams
Qbigrams=list(Qbigram[emj_words]) #make list
QmodelB = models.Word2Vec(Qbigrams, size=100, window=10, min_count=5, workers=8) #train

In [75]:
most_sim_emj(QmodelB.most_similar("up",topn=200))

['🆙',
 '😑',
 '🙄',
 '😒',
 '😤',
 '🙄😒',
 '😐',
 '🙄🙄🙄',
 '😴',
 '😩😂',
 '🙄🙄',
 '💯',
 '😂💯',
 '👌🏽',
 '🤗',
 '😏',
 '😩',
 '😴😴',
 '😭👐🏼🐵',
 '🙃',
 '😅',
 '🤔😂',
 '😫',
 '👌🏾',
 '🤔',
 '😕',
 '😌',
 '😂😂',
 '😂😴',
 '😂',
 '😡',
 '😂💀',
 '🤔🤔🤔',
 '🙃🙃',
 '🤑',
 '😡😡😡😡',
 '😒🙄',
 '🔫',
 '💁🏽',
 '😈',
 '🤔🤔',
 '🖕🏽',
 '💰💰💰',
 '😒😒',
 '😂🙄',
 '😒😒😒',
 '😡😡',
 '😑😑',
 '🙂',
 '😒😂',
 '😩😩😩',
 '🔫🔫🔫',
 '🤕',
 '🙄🙄🙄🙄',
 '👐🏾',
 '😛',
 '💅🏾',
 '😂😂😂',
 '👀',
 '😴😴😴',
 '😪',
 '😭😂',
 '💯💯',
 '😷',
 '😡😡😡',
 '😎',
 '🏃🏽',
 '🤐',
 '💯💯💯',
 '😑😒',
 '😓',
 '😒😑',
 '🙄😴',
 '💀💀💀',
 '👌🏼',
 '💀',
 '😈😈',
 '💀💀',
 '💸💸',
 '😂😂😂😂😂',
 '😭',
 '🔥🔥🔥',
 '😑😑😑',
 '💖✨💖✨',
 '💩',
 '💯👌',
 '😶',
 '🙄😂',
 '😂😭',
 '💪🏽',
 '😤😤',
 '😂😂😂😂',
 '👏🏾💪🏾',
 '😩😭',
 '🙄😤',
 '😑🙄',
 '😐😐😐',
 '😳',
 '💁🏾',
 '💸💰',
 '🖕🏼',
 '💂🏾',
 '💰💰',
 '🙄🙄🙄🙄🙄',
 '😂😩',
 '😅😅😅',
 '💸',
 '🙌🏽',
 '⬆️',
 '👐🏽',
 '😩😩',
 '😳😳😳😳',
 '🙃🙃🙃🙃',
 '💩💩',
 '💯😴',
 '‼️😝',
 '👐🏼',
 '🚮🚮',
 '🤔💯',
 '😝',
 '😞',
 '🙃🙃🙃',
 '🤔🙄',
 '😟',
 '🔥🔥🔥🔥🔥🔥🔥',
 '😂😂😂💀',
 '😠',
 '😂🤔',
 '😡😡😡😡😡',
 '😅😅']

In [78]:
QmodelB.save('data/Emoji2vec_B')